In [6]:
!pip -q install azure-storage-blob pandas openpyxl
!pip install python-dotenv

In [11]:
import os, io, re
from pathlib import Path
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

# --- Cargar variables del .env ---
load_dotenv()
CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
CONTAINER = os.getenv("AZURE_CONTAINER_NAME")
if not CONNECTION_STRING or not CONTAINER:
    raise RuntimeError("Faltan variables en .env: AZURE_STORAGE_CONNECTION_STRING o AZURE_CONTAINER_NAME")

# --- Parámetros configurables ---
PREFIX = os.getenv("BLOB_PREFIX", "raw/")  # p.ej. "raw/2025-08-21/" si quieres filtrar por fecha
# Carpeta de salida local (por defecto, dentro del proyecto). Cambia a D:\... si prefieres.
OUT_DIR = Path(os.getenv("EXPORT_DIR", Path.cwd() / "Datasets_csv")).resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

# --- Utilidades ---
def slugify(name: str) -> str:
    s = Path(name).stem
    s = s.replace(" ", "_")
    s = re.sub(r"[^0-9A-Za-z_]", "", s)
    return s.lower()

# --- Cliente Azure ---
svc = BlobServiceClient.from_connection_string(CONNECTION_STRING)
client = svc.get_container_client(CONTAINER)

# --- Descarga y conversión ---
reg = []
dfs = {}

for b in client.list_blobs(name_starts_with=PREFIX):
    name_lower = b.name.lower()
    if not name_lower.endswith((".xlsx", ".xls")):
        continue

    # Descargar el blob en memoria
    data = client.download_blob(b.name).readall()

    # Leer Excel (primera hoja por defecto)
    try:
        df = pd.read_excel(io.BytesIO(data))  # engine auto (openpyxl para .xlsx)
    except Exception as e:
        reg.append({"blob": b.name, "nombre_distintivo": None, "filas": None, "csv": None, "error": str(e)})
        continue

    key = slugify(Path(b.name).name)     # p.ej. "hotel_torre_mar"
    out_csv = OUT_DIR / f"{key}.csv"
    df.to_csv(out_csv, index=False, encoding="utf-8")

    dfs[key] = df
    reg.append({"blob": b.name, "nombre_distintivo": key, "filas": len(df), "csv": str(out_csv)})

# Resumen
pd.DataFrame(reg)


,blob,nombre_distintivo,filas,csv
0,raw/2025-08-21/Cosmos_Pacifico_Hotel.xlsx,cosmos_pacifico_hotel,1612,D:\Proyecto De Grado\notebooks\Datasets_csv\co...
1,raw/2025-08-21/Hotel_Cordillera_Buenaventura.xlsx,hotel_cordillera_buenaventura,2955,D:\Proyecto De Grado\notebooks\Datasets_csv\ho...
2,raw/2025-08-21/Hotel_Maguipi.xlsx,hotel_maguipi,1550,D:\Proyecto De Grado\notebooks\Datasets_csv\ho...
3,raw/2025-08-21/Hotel_Steven_Buenaventura.xlsx,hotel_steven_buenaventura,613,D:\Proyecto De Grado\notebooks\Datasets_csv\ho...
4,raw/2025-08-21/Hotel_Torre_Mar.xlsx,hotel_torre_mar,888,D:\Proyecto De Grado\notebooks\Datasets_csv\ho...
